#### create entity 

How to do the maximal match

1. tokenization phrase,  
2. embedding
3. do a classification on the text data
4. 

In [15]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.matcher import Matcher
import numpy as np
#nlp = spacy.load('en_core_web_sm')

In [16]:
NCIT_table=pd.read_pickle('./Data/NCIT_table.pickle')

use the semantic type as entity types

, synonyms , the pipeline accept theurapeutic or preventive procedure

In [143]:
NCIT_df['semantic type'].value_counts().to_csv('semantic_count.csv')

In [164]:
#semantics_ignore=['Qualitative Concept']


In [165]:
#NCIT_df[NCIT_df['semantic type'].isin(semantics_ignore)].sample(n=100)

In [17]:
#NCIT_df

NCIT_df=NCIT_table.reset_index().drop_duplicates(['code', 'concept name', 'parents', 'definition', 'semantic type',
       'synonyms'])

### prepare patterns

input: synonyms



output: tokenized synonyns 

In [18]:
NCIT_df.groupby('code')

In [19]:
synonymVC=NCIT_df['synonyms'].value_counts()
print ('# of synonym:',len(synonymVC))
print ('# of multi mapped:',(synonymVC>1).sum())

# of synonym: 388961
# of multi mapped: 13603


In [20]:
NCIT_df['synonym_word_len']=NCIT_df['synonyms'].str.len()

In [21]:
NCIT_df['synonyms_lower']=NCIT_df['synonyms'].str.lower()


In [22]:
#spacy.__version__

In [23]:
m1=NCIT_df.synonyms.isin(synonymVC.index[synonymVC==1])

m_test=NCIT_df.synonyms_lower.isin(['tp53','t cell','cell'])

m_text_length=NCIT_df.synonym_word_len>=3

"""
ascending=True: t, Cell 
"""
ncit_sub_df=NCIT_df[m_text_length].drop_duplicates(
    ['code','synonyms_lower']
     ).sort_values('n_th synonym').drop_duplicates('synonyms_lower')#.sort_values('synonym_word_len',ascending=True)#.sort_values('n_th synonym',ascending=False)

In [224]:
DEBUG=True

In [225]:
nlp = spacy.load('en_core_web_lg')
matcher = Matcher(nlp.vocab)

nerName='NCI'
nlp.vocab.strings.add(nerName)
EVENT = nlp.vocab.strings[nerName]

def add_event_ent(matcher, doc, i, matches):
    match_id, start, end = matches[i]
    entity = (EVENT, start, end)
    doc
    doc.ents += (entity,)
"""
for each synonyms, make a pattern 
"""
for _,tmpDf in  tqdm(ncit_sub_df.groupby('code')):
    
    #myEntityType=tmpDf['semantic type'].iloc[0]
    term_name=tmpDf['synonyms'].iloc[0]
    try:
        myPatterns=[]
        for  mySynonym in tmpDf['synonyms_lower'].tolist():
            mySynonymSplitted=mySynonym.split(' ')
            mySynonymSplitted=list(filter(lambda s: len(s)>0,mySynonymSplitted))
            ##add tokens if it doens't exist
            for myToken in mySynonymSplitted :
                if myToken not in  nlp.vocab.strings:
                    nlp.vocab.strings.add(myToken)              
            myPatterns.append([{'LEMMA':myToken} for myToken in mySynonymSplitted])
        args=[term_name,add_event_ent]+myPatterns
        matcher.add(*args)
        
    except ValueError:
        print ('failed ')

OSError: [E050] Can't find model 'en_core_web_lg'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [14]:
#!python -m spacy download en_core_web_sm

### tokenize the srs data


input: list of sentences, 
output: list of tokens

In [226]:
inS_dir='/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRS.pickle'

In [87]:
#pd.read_pickle
all_inS=pd.read_pickle(inS_dir)
#srxS=pd.read_pickle('/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRX.pickle')

In [190]:
###  test 
inS=all_inS.sample(n=10,random_state=2)#.unique()

In [206]:
len(inS)

10

In [207]:
docs=nlp.pipe(inS.tolist(),n_threads=2)

In [208]:
matches_L=[]
doc_L=[]
for doc in tqdm(docs):
    matches=matcher(doc)
    matches_L.append(matches)
    doc_L.append(doc)


0it [00:00, ?it/s]
2it [00:00, 13.55it/s]
4it [00:00, 14.12it/s]
8it [00:00, 20.74it/s]
10it [00:00, 24.73it/s]

In [209]:
NCIT_df[NCIT_df.synonyms_lower.isin(['bark'])]

,code,concept name,parents,definition,semantic type,n_th synonym,synonyms,synonym_word_len,synonyms_lower
270703,C51179,<http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...,C21381,Human GRK2 wild-type allele is located in the ...,Gene or Genome,3,BARK,4,bark
363783,C79615,<http://ncicb.nci.nih.gov/xml/owl/EVS/Thesauru...,C79611,NaN,Plant,0,Bark,4,bark


In [210]:
tmpDf

,0,1,2
0,12265437080716940157,0,1
1,9625438048578609369,2,3


In [220]:
tmpDf=pd.DataFrame(matches_L[1])
tmpDf[0].apply(lambda s:nlp.vocab.strings[s])


0    Human
1     Bark
Name: 0, dtype: object

In [221]:
nlp.vocab.strings[9625438048578609369]

In [222]:
tmpDoc=doc_L[1]
list(tmpDoc.noun_chunks)

[Human frontal cortex]

### build a word2vect 

for the data, tokenize 

In [ ]:
inS

## scratch

In [28]:
specieS=sampleS[sampleS.index.get_level_values(1)=='SCIENTIFIC_NAME']

In [29]:
selectedSrs=specieS[specieS=='Mus musculus'].index.get_level_values(0).unique()

In [75]:
#sampleS

In [30]:
%%time
mouseSrsS=sampleS[sampleS.index.get_level_values(0).isin(selectedSrs)]

CPU times: user 2.27 s, sys: 256 ms, total: 2.52 s
Wall time: 2.52 s


In [77]:
#sampleS

In [78]:
mouseAttribVC=mouseSrsS.index.get_level_values(1).value_counts()

In [80]:
#pd.set_option('display.max_rows', 100)

#mouseSrsS

In [33]:
g=mouseSrsS.groupby(level=[0,1])

In [49]:
mouseSrsS.head()

ERS949109  SCIENTIFIC_NAME              Mus musculus
           SUBMITTER_ID          IFMX1_1D-sc-2454762
           strain                                   
           sample_description    Lin_neg_bone marrow
           ArrayExpress-Sex                   female
dtype: object

In [50]:
inL=mouseSrsS.sample(n=1000).tolist()
for i,send in enumerate(inL):
    doc=nlp(send)
    matches=matcher(doc)
    for match in matches:
        print (doc[match[1]:match[2]],'        ',nlp.vocab.strings[match[0]])
    if i>10:
        break
    

DNA          DNA
Isolation          Isolation
Kit          Kit
Brain          Brain
not          Negation
determined          Determine
hypothalamus          Hypothalamus
whole          Whole
genome          Genome
sequence          Sequence
strains          Strain
Mus          Mouse
Mus musculus          Mus musculus


In [52]:
#NCIT_df

In [62]:
#NCIT_df[NCIT_df.synonyms.isin(['Mus musculus','Mouse'])]['definition'].values

Screen 12 Lmnb2 Knock-out Embryonic Stem Cell Derived Macrophages stimulated with Salmonella

In [198]:
list(doc.sents)[0]

Streptococcus pneumoniae
Streptococcus pneumoniae
SN20648-sc-2011-11-03T19:59:41Z-1101437


not applicable
Streptococcus pneumoniae
Pathogen sample from Salmonella enterica
Salmonella enterica subsp.

### scratch

In [171]:
mySynonymSplitted

['percutaneous',
 'coronary',
 'intervention',
 '(pci)',
 'for',
 'high',
 'risk',
 'non-st',
 'elevation',
 'myocardial',
 'infarction',
 'or',
 'unstable',
 'angina']

In [177]:
doc = nlp(u'glioblastoma cell line TP53 t cell')
matches = matcher(doc)


In [178]:
##word t probably map 
#lexem=doc.vocab['t']
#lexem.text

In [179]:
for match in matches:
    print (doc[match[1]:match[2]],nlp.vocab.strings[match[0]])

glioblastoma Glioblastoma
cell Cell
cell line Cell Line
line Long Interspersed Element
line Line
TP53 TP53 Gene
t cell T-Lymphocyte
cell Cell


In [ ]:
## 

In [180]:
match

(10878580257448234319, 5, 6)

In [176]:
for a in doc.ents:
    print (a,a.label_)

TP53 NCI
t NCI
cell NCI


In [167]:
doc

TP53 t cell

In [30]:

#EVENT = 

In [267]:
def f(a,b,c):
    print (a,b,c)
f(*args)

TP53 <function add_event_ent at 0x2b99bc527c80> [{'LEMMA': 'tp53'}]


In [268]:
myPatterns

[[{'LEMMA': 'tp53'}]]

In [269]:
myEntityType

'Gene or Genome'

In [118]:
EVENT

8984921151469646652

In [101]:
doc.ents

(tp53, t cell)

### scratch

In [16]:
myFirst=NCIT_df.groupby('code')['synonyms'].first()

In [ ]:
#myVC=myFirst.value_counts()

In [ ]:
#(myVC[myVC>1].index.values)

In [ ]:
def OrthsToPatterns(L):
    L

In [ ]:
inSynonymS.iloc[0]

In [ ]:
sampleS=pd.read_pickle('/cellar/users/btsui/Data/nrnb01_nobackup/METAMAP/allSRS.pickle')


In [ ]:
sampleS.head()

In [ ]:
import spacy

nlp = spacy.load('en_core_web_md')

In [ ]:
from spacy.matcher import Matcher


In [ ]:
matcher=Matcher(nlp.vocab)

### 

In [ ]:
import numpy
import spacy
from spacy.attrs import ENT_IOB, ENT_TYPE

nlp = spacy.load('en_core_web_sm')
doc = nlp.make_doc(u'London is a big city in the United Kingdom.')
print('Before', list(doc.ents))  # []

header = [ENT_IOB, ENT_TYPE]
attr_array = numpy.zeros((len(doc), len(header)))
attr_array[0, 0] = 3  # B
attr_array[0, 1] = doc.vocab.strings[u'GPE']
doc.from_array(header, attr_array)
print('After', list(doc.ents))  # [London]

In [ ]:
attr_array

In [ ]:
import spacy

nlp = spacy.load('en')

def merge_phrases(matcher, doc, i, matches):
    '''
    Merge a phrase. We have to be careful here because we'll change the token indices.
    To avoid problems, merge all the phrases once we're called on the last match.
    '''
    if i != len(matches)-1:
        return None
    spans = [(ent_id, label, doc[start : end]) for ent_id, label, start, end in matches]
    for ent_id, label, span in spans:
        span.merge('NNP' if label else span.root.tag_, span.text, nlp.vocab.strings[label])
    
    matcher.add()
matcher.add('GoogleNow', merge_phrases, [{ORTH: 'Google'}, {ORTH: 'Now'}])


In [ ]:
def on_match(matcher, doc, id, matches):
    print('Matched!', matches)

matcher = Matcher(nlp.vocab)
matcher.add('HelloWorld', on_match, [{'LOWER': 'hello'}, {'LOWER': 'world'}])
matcher.add('GoogleMaps', on_match, [{'ORTH': 'Google'}, {'ORTH': 'Maps'}])
doc = nlp(u'HELLO WORLD on Google Maps.')
matches = matcher(doc)

In [ ]:
e=doc.ents[0]

https://spacy.io/usage/linguistic-features#on_match

Adding on_match rules

[(3492015959717556363, 5, 9), (3492015959717556363, 5, 10)]
[(3492015959717556363, 5, 9), (3492015959717556363, 5, 10)]


In [213]:
##entity label, 

In [214]:
ent=doc.ents[0]

In [ ]:
ent.label_

In [ ]:
len(matches)

In [ ]:
matches[0]

### 

using a function to define different entity in princ

In [105]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

EVENT = nlp.vocab.strings['cell']
def add_event_ent(matcher, doc, i, matches):
    # Get the current match and create tuple of entity label, start and end.
    # Append entity to the doc's entity. (Don't overwrite doc.ents!)
    #print (matches)
    match_id, start, end = matches[i]
    entity = (EVENT, start, end)
    doc.ents += (entity,)

    
matcher.add('cell', add_event_ent,
            [{'ORTH': 'cell'}])
EVENT = nlp.vocab.strings['google']
def add_event_ent(matcher, doc, i, matches):
    # Get the current match and create tuple of entity label, start and end.
    # Append entity to the doc's entity. (Don't overwrite doc.ents!)
    #print (matches)
    match_id, start, end = matches[i]
    entity = (EVENT, start, end)
    doc.ents += (entity,)
matcher.add('GoogleIO', add_event_ent,
            [{'ORTH': 'google'}])
           # [{'ORTH': 'Google'}, {'ORTH': 'I'}, {'ORTH': '/'}, {'ORTH': 'O'}, {'IS_DIGIT': True}])


In [106]:
doc = nlp(u"cell is google. 10 days")


In [ ]:
matcher(doc)

In [ ]:
#nlp.vocab.strings['Therapeutic or Preventive Procedure']

In [ ]:
my_ent=doc.ents[2]

In [ ]:
nlp.vocab.strings['cell']


In [64]:
#!pip install gensim